In [10]:
import pandas as pd
import numpy as np
import spacy
from spacy_sentiws import spaCySentiWS
import re
import swifter
import time
import statistics
import dask.dataframe as dd
import multiprocessing as mp

In [11]:
df = pd.read_pickle('../../data/p_comment.pkl.zip')
ddf = dd.from_pandas(df, npartitions=mp.cpu_count())
print(df.shape)
df.head()

(421572, 11)


,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,Headline,Comment,createdAt,VoteNegative,VotePositive,p_merged_text
0,1049737336,-1,2000114365338,568012,1.0,NaN,Das Virus ist wohl doch VIEL ansteckender als ...,2020-02-10 08:48:31.513,False,True,Virus wohl ansteckend denken wohl wirklich Luf...
1,1049737620,1049737336,2000114365338,505220,1.0,NaN,"So ist es, somit sind bestimmt auch deutlich m...",2020-02-10 08:59:34.357,False,True,somit bestimmen deutlich mehr Mensch China inf...
2,1049737708,-1,2000114365338,508661,1.0,NaN,Das ist für mich eine Horrorvorstellung. Final...,2020-02-10 09:03:20.527,False,True,Horrorvorstellung Final Destination x
3,1049737722,-1,2000114365338,583906,1.0,NaN,"Wäre interessant zu wissen, ob vor allem schon...",2020-02-10 09:03:48.857,True,False,sein interessant wissen schon krank schwächen ...
4,1049737801,1049737746,2000114365338,635467,1.0,NaN,Besser als Häfn! ;-),2020-02-10 09:07:10.360,False,True,gut Häfn


In [14]:
# exclude the found phrases _ and only consider single words
text = 'Monat dauern Traumtänzer stehen_einfach dafür'.replace('_', ' ');
nlp = spacy.load('de_core_news_md')
nlp.add_pipe('sentiws',config={'sentiws_path': '../../data/sentiws/'})
doc = nlp(text)
for token in doc:
    print('{}, {}, {}'.format(token.text, token._.sentiws, token.pos_))

Monat, None, NOUN
dauern, None, VERB
Traumtänzer, None, NOUN
stehen, None, VERB
einfach, None, ADV
dafür, None, ADV


In [16]:
def get_overall_sentiment(text):
    text = text.replace('_',' ')
    doc = nlp(text)
    sentiment_score = 0
    for token in doc:
        #print('{}, {}, {}'.format(token.text, token._.sentiws, token.pos_))
        sentiment_score += token._.sentiws or 0
    
    return sentiment_score / len(doc)

start = time.time()
df['sentiment'] = df['p_merged_text'].swifter.apply(lambda x: get_overall_sentiment(str(x)))
#df['sentiment'] = ddf.map_partitions(lambda df: df.apply(lambda row: get_overall_sentiment(row['p_merged_text']),axis=1), meta=("sentiment", float)).compute(scheduler="processes")

end = time.time()

print('processing time: ', end-start)

df.head()

C:\Users\e11925939\Anaconda3\envs\env39\lib\site-packages\swifter\swifter.py:87: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  warnings.warn(


Pandas Apply:   0%|          | 0/421572 [00:00<?, ?it/s]

processing time:  5117.059106349945


,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,Headline,Comment,createdAt,VoteNegative,VotePositive,p_merged_text,sentiment
0,1049737336,-1,2000114365338,568012,1.0,NaN,Das Virus ist wohl doch VIEL ansteckender als ...,2020-02-10 08:48:31.513,False,True,Virus wohl ansteckend denken wohl wirklich Luf...,0.000138
1,1049737620,1049737336,2000114365338,505220,1.0,NaN,"So ist es, somit sind bestimmt auch deutlich m...",2020-02-10 08:59:34.357,False,True,somit bestimmen deutlich mehr Mensch China inf...,0.000000
2,1049737708,-1,2000114365338,508661,1.0,NaN,Das ist für mich eine Horrorvorstellung. Final...,2020-02-10 09:03:20.527,False,True,Horrorvorstellung Final Destination x,0.000000
3,1049737722,-1,2000114365338,583906,1.0,NaN,"Wäre interessant zu wissen, ob vor allem schon...",2020-02-10 09:03:48.857,True,False,sein interessant wissen schon krank schwächen ...,0.000000
4,1049737801,1049737746,2000114365338,635467,1.0,NaN,Besser als Häfn! ;-),2020-02-10 09:07:10.360,False,True,gut Häfn,0.000000


In [17]:
#weight the sentiment score based on the votes
# less negative than possitive comments
df1 = df.copy()
wh=2
wl=1
#df1.to_csv('../../data/feature/sentiws_comment.csv', encoding='utf-8', index=False)
df1['weight'] = np.where((df1['sentiment'] > 0) & (df1['VotePositive']), wh,
                        np.where((df1['sentiment'] < 0) & (df1['VotePositive']), wh, wl))
df1.head()

,ID_Posting,ID_Posting_Parent,ID_GodotObject,ID_CommunityIdentity,id_posting_status,Headline,Comment,createdAt,VoteNegative,VotePositive,p_merged_text,sentiment,weight
0,1049737336,-1,2000114365338,568012,1.0,NaN,Das Virus ist wohl doch VIEL ansteckender als ...,2020-02-10 08:48:31.513,False,True,Virus wohl ansteckend denken wohl wirklich Luf...,0.000138,2
1,1049737620,1049737336,2000114365338,505220,1.0,NaN,"So ist es, somit sind bestimmt auch deutlich m...",2020-02-10 08:59:34.357,False,True,somit bestimmen deutlich mehr Mensch China inf...,0.000000,1
2,1049737708,-1,2000114365338,508661,1.0,NaN,Das ist für mich eine Horrorvorstellung. Final...,2020-02-10 09:03:20.527,False,True,Horrorvorstellung Final Destination x,0.000000,1
3,1049737722,-1,2000114365338,583906,1.0,NaN,"Wäre interessant zu wissen, ob vor allem schon...",2020-02-10 09:03:48.857,True,False,sein interessant wissen schon krank schwächen ...,0.000000,1
4,1049737801,1049737746,2000114365338,635467,1.0,NaN,Besser als Häfn! ;-),2020-02-10 09:07:10.360,False,True,gut Häfn,0.000000,1


In [18]:
## calculate weighterd median of the discussion sentiment
df2 = df1[['ID_Posting', 'ID_Posting_Parent','ID_GodotObject', 'createdAt', 'sentiment', 'weight']].copy()
df2['sentiment'] = df2['sentiment'] * df2['weight']
df2.drop('weight', inplace=True, axis=1)

# method for calculating median of subconversation
def conversation_sentiment_scores(posting_id, sents, replies_df):
    # find all direct replies for the posting
    #print(posting_id)
    replies = replies_df[replies_df["ID_Posting_Parent"] == posting_id]
    #print(len(replies))
    if replies.empty:
        # Return the current depth if there are no replies
        return sents
    else:
        #sents.extend(replies['sentiment'])
        for row in replies.itertuples():
            sub = conversation_sentiment_scores(row.ID_Posting, [row.sentiment], replies_df)
            sents += sub
            
        return sents

    
    
# First, we need to filter out only the root comments
root_comments = df2[df2['ID_Posting_Parent'] == -1]

# Next, we need to filter out only the replies
replies = df2[df2['ID_Posting_Parent'] != -1]

#conversation_sentiment_scores(1052015173, [3], replies)
root_comments['overal_sentiment'] = root_comments.swifter.apply(lambda x: statistics.median(
    conversation_sentiment_scores(x.ID_Posting, [x.sentiment], replies)), axis=1)

root_comments.reset_index(drop=True, inplace=True)
print(root_comments.shape)
root_comments.head()

C:\Users\e11925939\Anaconda3\envs\env39\lib\site-packages\swifter\swifter.py:87: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  warnings.warn(


Pandas Apply:   0%|          | 0/127612 [00:00<?, ?it/s]

(127612, 6)


C:\Users\e11925939\AppData\Local\Temp\8\ipykernel_68728\1109749256.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  root_comments['overal_sentiment'] = root_comments.swifter.apply(lambda x: statistics.median(


,ID_Posting,ID_Posting_Parent,ID_GodotObject,createdAt,sentiment,overal_sentiment
0,1049737336,-1,2000114365338,2020-02-10 08:48:31.513,0.000276,0.000000
1,1049737708,-1,2000114365338,2020-02-10 09:03:20.527,0.000000,0.000000
2,1049737722,-1,2000114365338,2020-02-10 09:03:48.857,0.000000,0.000000
3,1049738435,-1,2000114365338,2020-02-10 09:29:54.117,0.082933,0.082933
4,1049738462,-1,2000114365338,2020-02-10 09:30:39.000,0.000000,0.000000


In [19]:
root_comments.to_csv('../../data/feature/sentiws_comment.csv', encoding='utf-8', index=False)